In [19]:
import dotenv

dotenv.load_dotenv()

True

In [20]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [21]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Analyze the every following code.
After anylyzing try to find some vulnerabilities in the code.
Here is a list of i wanna find: 
    - Reentrency Attack: Reentrancy is a vulnerability that allows an attacker to re-enter a function multiple times before the first function call is finished. so whenever the contract makes external call to other addresses, this is a possibility for reentrancy attack.This can lead to unexpected behavior, including reordering of transactions, and can be used to drain funds from a contract. 
    - Replay signatures attacks: The original account will sign a message then the delivery account will send the message to a smart contract, that way it is the delivery account that pays for the transaction fees and not the original account.
If you find anything, please provide a list of them in JSON format. Escape quotes in the JSON with a backslash. 
The JSON should contain the following fields:
- "line": The line number where the issue occurs.
- "token": The specific code token that is problematic.
- "problem": A short name for the issue.
- "severity": The severity of the issue (low, high).
- "explanation": A brief explanation of why this is a security risk.
- "migration": A brief explanation of how to fix this issue.
Code:
{code}
"""
)

In [22]:
import os

files = os.listdir('snippets')
code_snippets = []
for file in files:
    with open(f'snippets/{file}', 'r') as f:
        code_snippets.append(f.read())
print(code_snippets)

['// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank{\n    mapping(address => uint) public credit;\n    \n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public{\n        if (credit[msg.sender]>= amount) {\n            require(msg.sender.call.value(amount)());\n            credit[msg.sender]-=amount;\n        }\n    }\n}', '// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank is ReentrancyGuard {\n    mapping(address => uint) public credit;\n\n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public nonReentrant {\n        require(credit[msg.sender] >= amount, "Insufficient balance");\n        \n        credit[msg.sender] -= amount; // Effect: Update state before transferring funds\n        \n        (bool success,) = payable(msg.sender).call{value: amount}(""); // Interaction: T

In [23]:
from langchain.chains import LLMChain
chain = chain = prompt | model

In [28]:
code = code_snippets[1]

In [29]:
result = chain.invoke({"code": code})
result

AIMessage(content='Here\'s the analysis of the provided smart contract along with potential vulnerabilities identified in JSON format:\n\n```json\n[\n    {\n        "line": 12,\n        "token": "credit[msg.sender] -= amount;",\n        "problem": "Reentrancy Attack",\n        "severity": "high",\n        "explanation": "The function updates the state (account balance) before making an external call (transfer). Although the nonReentrant modifier is applied, if it were absent or bypassed, reentrancy could lead to multiple withdrawals before the state change.",\n        "migration": "Ensure all state changes are completed before any external calls or use established patterns like \'checks-effects-interactions\' or consider implementing a reentrancy guard."\n    }\n]\n```\n\n**Explanation of the JSON fields:**\n- **line**: The line number where the issue occurs.\n- **token**: The specific code token that is problematic.\n- **problem**: A short name for the issue.\n- **severity**: The seve

In [30]:
result_text = result.content.strip('`json')
print(code)
print(result_text)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract PiggyBank is ReentrancyGuard {
    mapping(address => uint) public credit;

    function deposit() public payable {
        credit[msg.sender] += msg.value;
    }

    function withdraw(uint amount) public nonReentrant {
        require(credit[msg.sender] >= amount, "Insufficient balance");
        
        credit[msg.sender] -= amount; // Effect: Update state before transferring funds
        
        (bool success,) = payable(msg.sender).call{value: amount}(""); // Interaction: Transfer funds last
        require(success, "Transfer failed");
    }
}

Here's the analysis of the provided smart contract along with potential vulnerabilities identified in JSON format:

```json
[
    {
        "line": 12,
        "token": "credit[msg.sender] -= amount;",
        "problem": "Reentrancy Attack",
        "severity": "high",
        "explanation": "The function updates the state (account balance) before making an external call (

In [27]:
import json
(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])
json.loads(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])

[{'line': 12,
  'token': 'msg.sender.call.value(amount)()',
  'problem': 'Reentrancy Attack',
  'severity': 'high',
  'explanation': "This line calls an external function (msg.sender) which can potentially allow the attacker to re-enter the withdraw function before the credit is reduced, possibly leading to multiple withdrawals and draining the contract's funds.",
  'migration': "Use the 'checks-effects-interactions' pattern. First, update the credit mapping to reflect the withdrawal before making the external call. Alternatively, use 'transfer' or 'send' for sending ether instead of 'call'."},
 {'line': 5,
  'token': 'pragma solidity ^0.8.0;',
  'problem': 'Replay signatures attack (Indirect)',
  'severity': 'medium',
  'explanation': 'While not directly addressed in this contract, using a versioned pragma might allow for potential exploitations if not combined with measures like nonces or specific replay protection mechanisms.',
  'migration': 'Consider implementing a nonce mechanism